<a href="https://colab.research.google.com/github/viruszig-cyber/SasolCustomerRetentionRecruitmentCompetition/blob/main/sasol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [126]:
import numpy as np
import pandas as pd

from sklearn import svm

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

import pickle

from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb

data=pd.read_csv('Train.csv')

data = pd.concat(
    [
        pd.read_csv('Train.csv'),
        pd.read_csv('Test.csv')
    ],
    axis=0)
data.reset_index(drop=True, inplace=True)
data

In [164]:
# printing the first 5 rows of the dataset
data=pd.read_csv('/content/Test.csv')
data

,ID,region,tenure,Amount,refill_frequency,revenue,arpu_segment,frequency,data_volume,on_net,Procuct_1,Procuct_2,zone1,zone2,mrg,regularity,top_pack,freq_top_pack
0,ID_1I1XQIKT5A9L,Droidika,K > 24 month,2008.55,16.0,2010.26,670.09,17.0,10.0,247.0,17.0,70.0,NaN,NaN,NO,4.0,On net 200F=Unlimited _call24H,11.0
1,ID_KS81OHJGUETH,Droidika,K > 24 month,256.41,2.0,250.43,83.33,3.0,NaN,15.0,100.0,84.0,NaN,NaN,NO,7.0,On net 200F=Unlimited _call24H,1.0
2,ID_A84OW2TIMWBV,Nebulith,K > 24 month,1581.20,8.0,1578.21,526.07,8.0,436.0,47.0,113.0,5.0,NaN,NaN,NO,46.0,Jokko_Daily,6.0
3,ID_BM9GONF7TJCO,Mandalore,K > 24 month,1431.62,8.0,1432.05,477.35,10.0,3262.0,22.0,25.0,NaN,0.0,NaN,NO,57.0,On net 200F=Unlimited _call24H,3.0
4,ID_5C7R4UZQFWF3,Mandalore,K > 24 month,2008.55,18.0,1793.59,597.86,24.0,NaN,70.0,23.0,2.0,NaN,NaN,NO,44.0,Twter_U2opia_Daily,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328011,ID_D86YU3N6W4OE,Mandalore,K > 24 month,3418.80,14.0,3964.96,1321.79,25.0,NaN,17.0,77.0,NaN,0.0,0.0,NO,34.0,All-net 500F=2000F;5d,10.0
328012,ID_V5X0KZYFR1Z3,Luminous,K > 24 month,641.03,2.0,508.97,169.66,2.0,870.0,1.0,NaN,NaN,NaN,NaN,NO,62.0,"Data: 100 F=40MB,24H",1.0
328013,ID_K6E31P7IGBLC,Astral,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,46.0,NaN,NaN
328014,ID_4FBNTUM07OQJ,Stellaris,K > 24 month,6581.20,5.0,6579.91,2193.16,11.0,3500.0,477.0,71.0,36.0,117.0,NaN,NO,62.0,MIXT: 200mnoff net _unl on net _5Go;30d,3.0


In [165]:
data.isnull().sum()
#data['Target'].fillna(0,inplace=True)

ID                       2
region              116687
tenure                   1
Amount              103029
refill_frequency    103029
revenue              98375
arpu_segment         98375
frequency            98375
data_volume         148425
on_net              107574
Procuct_1           123081
Procuct_2           183392
zone1               298492
zone2               304137
mrg                      1
regularity               1
top_pack            124338
freq_top_pack       124338
dtype: int64

In [129]:
#data.drop(columns='ID', inplace=True) # removing unnecessary colum

#data.columns = [col.lower().replace(' ', '_') for col in data.columns] # renaming the column names

In [166]:
#data.dropna(thresh = data.shape[1]/3, axis = 0, inplace = True)
#data = data[data.isnull().sum(axis=1) < 13]
df=data
# Handle missing numerical values
#df['tenure'].fillna(df['tenure'].median(), inplace=True)

# replacing values
from sklearn import preprocessing

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder


for col in ["region","top_pack"]:
    df[col] = LabelEncoder().fit_transform(df[col])



le = LabelEncoder()
#le.fit_transform(df['region'])
#df['region'].fillna(df['region'].mode()[0], inplace=True)

#le.fit_transform(df['top_pack'])
#df['top_pack'].fillna(df['top_pack'].mode()[0], inplace=True)



df['tenure'].replace(['K > 24 month', 'H 15-18 month','I 18-21 month','H 15-18 month','G 12-15 month','K > 24 month','J 21-24 month','D 3-6 month','E 6-9 month','F 9-12 month'],
                        [0, 1,2,3,4,5,6,7,8,9], inplace=True)

df['mrg'].replace(['NO', 'YES '],
                        [0, 1], inplace=True)


df['Amount'].fillna(df['Amount'].mean(), inplace=True)
df['refill_frequency'].fillna(df['refill_frequency'].mode()[0], inplace=True)
df['revenue'].fillna(df['revenue'].mean(), inplace=True)
df['arpu_segment'].fillna(df['arpu_segment'].mean(), inplace=True)
df['frequency'].fillna(df['frequency'].mode()[0], inplace=True)
df['data_volume'].fillna(df['data_volume'].mean(), inplace=True)
df['on_net'].fillna(0, inplace=True)
df['Procuct_1'].fillna(df['Procuct_1'].mode()[0], inplace=True)
df['Procuct_2'].fillna(df['Procuct_2'].mode()[0], inplace=True)
df['zone1'].fillna(df['zone1'].mode()[0], inplace=True)
df['zone2'].fillna(df['zone2'].mode()[0], inplace=True)
df['freq_top_pack'].fillna(df['freq_top_pack'].mode()[0], inplace=True)


# You can handle other categorical columns in a similar manner

# Drop rows with missing values
#df.dropna(inplace=True)

# Validate and verify the data
print(df.isnull().sum())
print(df.head())

ID                  2
region              0
tenure              1
Amount              0
refill_frequency    0
revenue             0
arpu_segment        0
frequency           0
data_volume         0
on_net              0
Procuct_1           0
Procuct_2           0
zone1               0
zone2               0
mrg                 1
regularity          1
top_pack            0
freq_top_pack       0
dtype: int64
                ID  region  tenure   Amount  refill_frequency  revenue  \
0  ID_1I1XQIKT5A9L       2     5.0  2008.55              16.0  2010.26   
1  ID_KS81OHJGUETH       2     5.0   256.41               2.0   250.43   
2  ID_A84OW2TIMWBV      10     5.0  1581.20               8.0  1578.21   
3  ID_BM9GONF7TJCO       8     5.0  1431.62               8.0  1432.05   
4  ID_5C7R4UZQFWF3       8     5.0  2008.55              18.0  1793.59   

   arpu_segment  frequency  data_volume  on_net  Procuct_1  Procuct_2  zone1  \
0        670.09       17.0    10.000000   247.0       17.0       7

In [167]:
df

,ID,region,tenure,Amount,refill_frequency,revenue,arpu_segment,frequency,data_volume,on_net,Procuct_1,Procuct_2,zone1,zone2,mrg,regularity,top_pack,freq_top_pack
0,ID_1I1XQIKT5A9L,2,5.0,2008.550000,16.0,2010.260000,670.09000,17.0,10.000000,247.0,17.0,70.0,0.0,0.0,0.0,4.0,75,11.0
1,ID_KS81OHJGUETH,2,5.0,256.410000,2.0,250.430000,83.33000,3.0,3347.687607,15.0,100.0,84.0,0.0,0.0,0.0,7.0,75,1.0
2,ID_A84OW2TIMWBV,10,5.0,1581.200000,8.0,1578.210000,526.07000,8.0,436.000000,47.0,113.0,5.0,0.0,0.0,0.0,46.0,55,6.0
3,ID_BM9GONF7TJCO,8,5.0,1431.620000,8.0,1432.050000,477.35000,10.0,3262.000000,22.0,25.0,1.0,0.0,0.0,0.0,57.0,75,3.0
4,ID_5C7R4UZQFWF3,8,5.0,2008.550000,18.0,1793.590000,597.86000,24.0,3347.687607,70.0,23.0,2.0,0.0,0.0,0.0,44.0,89,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328011,ID_D86YU3N6W4OE,8,5.0,3418.800000,14.0,3964.960000,1321.79000,25.0,3347.687607,17.0,77.0,1.0,0.0,0.0,0.0,34.0,12,10.0
328012,ID_V5X0KZYFR1Z3,7,5.0,641.030000,2.0,508.970000,169.66000,2.0,870.000000,1.0,1.0,1.0,0.0,0.0,0.0,62.0,17,1.0
328013,ID_K6E31P7IGBLC,0,5.0,2361.848505,1.0,2350.674861,783.56101,1.0,3347.687607,0.0,1.0,1.0,0.0,0.0,0.0,46.0,101,1.0
328014,ID_4FBNTUM07OQJ,13,5.0,6581.200000,5.0,6579.910000,2193.16000,11.0,3500.000000,477.0,71.0,36.0,117.0,0.0,0.0,62.0,59,3.0


In [168]:
df['regularity'].replace(['NO', 'YES '],
                        [0, 1], inplace=True)
df['freq_top_pack'].replace(['NO', 'All-net 500F=2000F;5d'],
                        [0, 1], inplace=True)

In [171]:
df.dropna(inplace=True)

In [172]:
df.isnull().sum()

ID                  0
region              0
tenure              0
Amount              0
refill_frequency    0
revenue             0
arpu_segment        0
frequency           0
data_volume         0
on_net              0
Procuct_1           0
Procuct_2           0
zone1               0
zone2               0
mrg                 0
regularity          0
top_pack            0
freq_top_pack       0
dtype: int64

In [173]:
df.apply(pd.to_numeric)

ValueError: ignored

In [ ]:
df.fillna(0,inplace=True)

In [134]:
df.isnull().sum()

region              0
tenure              3
Amount              0
refill_frequency    0
revenue             0
arpu_segment        0
frequency           0
data_volume         0
on_net              0
Procuct_1           0
Procuct_2           0
zone1               0
zone2               0
mrg                 3
regularity          3
top_pack            0
freq_top_pack       0
Target              0
dtype: int64

In [135]:
df.fillna(0,inplace=True)

In [174]:
df.to_csv('test_sample.csv', index=False)


In [137]:
df.rename(columns = {'Target': 'target'},inplace = True)

In [138]:
encoded_df = df.drop('target',axis = 1)
df_target= df[['target']]

In [139]:
import random
random.seed(123)
X_train, X_test, y_train, y_test = train_test_split(encoded_df, df_target, test_size=0.2, random_state=123)

In [140]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from sklearn.utils import check_array, _safe_indexing, sparsefuncs_fast, check_X_y, check_random_state
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
from sklearn.neighbors import NearestNeighbors
from sklearn.base import clone
from numbers import Integral
from sklearn.svm import SVC
from collections import Counter
import seaborn as sns
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, SMOTENC, SVMSMOTE
import os
# import missingpy as missingpy
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
import pickle
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, precision_recall_curve, f1_score, roc_auc_score, make_scorer
from sklearn.model_selection import cross_val_score, cross_validate

In [141]:
columns =['Amount','revenue','arpu_segment','data_volume','on_net','Procuct_2','zone1','zone2']
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()


In [142]:
X_train = pd.DataFrame(sc.fit_transform(X_train.values), columns=X_train.columns, index=X_train.index)

In [143]:
X_test = pd.DataFrame(sc.fit_transform(X_test.values), columns=X_test.columns, index=X_test.index)

In [144]:
X_train.shape

(128502, 17)

In [145]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score,accuracy_score,f1_score, cohen_kappa_score
from imblearn.metrics import sensitivity_score,specificity_score
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, KFold

In [146]:
# choosing best parameters
import random
random.seed(123)
params = {"alpha":np.arange(0.00001, 10, 500)}
kf=KFold(n_splits=5,shuffle=True, random_state=42)

In [147]:
## initializze the lasso model for feature selection
lasso = Lasso()

In [148]:
lasso_cv=GridSearchCV(lasso, param_grid=params, cv=kf)
lasso_cv.fit(X_train, y_train)
print("Best Params {}".format(lasso_cv.best_params_))

Best Params {'alpha': 1e-05}


In [149]:
# calling the model with the best parameter
lasso1 = Lasso(alpha=0.00001)
lasso1.fit(X_train, y_train)

Lasso(alpha=1e-05)

In [150]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train ,y_train)

KNeighborsClassifier()

In [151]:
knn_pred = knn_model.predict(X_test)

In [152]:
PRESC = precision_score(y_test, knn_pred)
ACC = accuracy_score(y_test, knn_pred)
F1 = f1_score(y_test, knn_pred)
KAPPA = cohen_kappa_score(y_test, knn_pred)
AUC = roc_auc_score(y_test, knn_pred)
SNS = sensitivity_score(y_test, knn_pred)
SPS = specificity_score(y_test, knn_pred)

In [153]:
print("PRECISION for SMOTE_ENC 5% : ", PRESC)
print("ACCURACY for SMPTE_ENC 5% : ", ACC)
print("F1 for SMOTE_ENC 5% : ", F1)
print("KAPPA for SMOTE_ENC 5% : ", KAPPA)
print("AUC for SMOTE_ENC 5% : ", AUC)
print("SENSITIVITY SMOTE_ENC 5% : ", SNS)
print("SPECIFICITY for SMOTE_ENC 5% : ", SPS)

PRECISION for SMOTE_ENC 5% :  0.9129961731822616
ACCURACY for SMPTE_ENC 5% :  0.8892797111373965
F1 for SMOTE_ENC 5% :  0.931893465065962
KAPPA for SMOTE_ENC 5% :  0.6367689053724557
AUC for SMOTE_ENC 5% :  0.7988529248701107
SENSITIVITY SMOTE_ENC 5% :  0.9515895671215735
SPECIFICITY for SMOTE_ENC 5% :  0.646116282618648


model training

In [155]:
classifier = svm.SVC(kernel='linear')
#training the support vector Machine Classifier
classifier.fit(X_train, y_train)

SVC(kernel='linear')

In [156]:
# accuracy score on the training data
X_train_prediction = classifier.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, y_train)

In [157]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.8882585485050817


In [158]:
# accuracy score on the test data
X_test_prediction = classifier.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, y_test)

In [159]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.8860735852580465


In [160]:
import pickle
filename = 'sasolTest.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [178]:
# loading the saved model
loaded_model = pickle.load(open('sasol.sav', 'rb'))

In [179]:
for column in df.columns:
  print(column)

ID
region
tenure
Amount
refill_frequency
revenue
arpu_segment
frequency
data_volume
on_net
Procuct_1
Procuct_2
zone1
zone2
mrg
regularity
top_pack
freq_top_pack


In [180]:
import numpy as np
import pandas as pd

# Load the CSV file
data = pd.read_csv('test_sample.csv')

# Define the columns to pick
columns_to_pick = ['region', 'tenure', 'Amount', 'refill_frequency',
                   'revenue','arpu_segment','frequency','data_volume','on_net','Procuct_1','Procuct_2',
                   'zone1','zone2','mrg','regularity','top_pack','freq_top_pack']

# Iterate through the rows and make predictions
predictions = []
for index, row in data.iterrows():
    input_data = row[columns_to_pick].values
    input_data_reshaped = np.reshape(input_data, (1, -1))
    prediction = loaded_model.predict(input_data_reshaped)
    predictions.append(prediction[0])

# Add the predicted results as a new column in the dataframe
data['Target'] = predictions

# Save the modified dataframe to a new CSV file
data.to_csv('sasol_output_file.csv', index=False)